## Experiment on influence of control genes selection in gene signature scoring in last experession bin of the CRC and ESCC dataset.
In this jupyter notebook we explored the influence, i.e., the bias, that control gene selection can have on signature scoring. We explore the absolute distance between the average expression of each gene in the last expression bin of the CRC and ESCC dataset and the mean value of the control genes' expressions.

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append('../..')
from data.load_data import load_datasets
from data.constants import BASE_PATH_EXPERIMENTS


from signaturescoring.utils.utils import get_mean_and_variance_gene_expression, get_bins_wrt_avg_gene_expression,get_least_variable_genes_per_bin_v1
sc.settings.verbosity = 0

In [ ]:
sns.set_style('ticks')

In [ ]:
plt.rcParams.update({'pdf.fonttype':42, 'font.family':'sans-serif', 'font.sans-serif':'Arial', 'font.size':14})

In [ ]:
save = True
show = True

Uncomment the desired data path and dataset. 

In [ ]:
## define the path to the preprocessed CRC dataset
dataset = 'luad' # 'crc' 'escc' 'luad'
adata = load_datasets(dataset)

### define path to the folder where data an visualizations should be stored
storing_path = os.path.join(BASE_PATH_EXPERIMENTS,'control_genes_selection')

In [ ]:
df_mean_var = get_mean_and_variance_gene_expression(
    adata,
    estim_var=True,
    show_plots=True,
    store_path=None,
)

In [ ]:
ctrl_size = 100
nr_bins=25

In [ ]:
gene_bins = get_bins_wrt_avg_gene_expression(df_mean_var['mean'], nr_bins)

In [ ]:
gene_bins.value_counts().sort_index()

In [ ]:
plt.rcParams.update({'pdf.fonttype':42, 'font.family':'sans-serif', 'font.sans-serif':'Arial', 'font.size':14})

In [ ]:
plt.figure(figsize=(10, 6))
for val in np.where(gene_bins.values[:-1] != gene_bins.values[1:])[0]:
    plt.axvline(val, c='g', lw=2)
plt.axvline(val, c='g', lw=2, label='expression bins')

plt.plot(df_mean_var['mean'].values, lw=3, label='sorted avg. gene epxression')
plt.legend(fontsize=14)
plt.title(f'Sorted average gene expression with expression bins of {dataset.upper()}',fontsize=16)
if save:
    plt.savefig(os.path.join(storing_path, f'avg_gene_epxression_{dataset}.svg'),format='svg')
    plt.savefig(os.path.join(storing_path, f'avg_gene_epxression_{dataset}.png'),format='png',dpi=300)
if show:
    plt.show()
else:
    plt.close()

In [ ]:
genes_last_bin = gene_bins[gene_bins==24].index.tolist()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(df_mean_var.loc[genes_last_bin,'mean'].values, lw=3, label='sorted avg. gene epxression')
plt.title(f'Sorted average gene expression of last expresion bin {dataset.upper()}',fontsize=16)
if save:
    plt.savefig(os.path.join(storing_path, f'avg_gene_epxression_in_last_bin_{dataset}.svg'),format='svg')
    plt.savefig(os.path.join(storing_path, f'avg_gene_epxression_in_last_bin_{dataset}.png'),format='png',dpi=300)
if show:
    plt.show()
else:
    plt.close()

In [ ]:
df_mean_var.loc[genes_last_bin,'mean']

In [ ]:
# We remove genes at the boeader for which ANS could not create a valid control set, since we only consider the last expression bin
sig_genes_to_test_for = genes_last_bin[50:-50]

In [ ]:
avg_exprs_last_expression_bin = df_mean_var.loc[genes_last_bin,'mean']

In [ ]:
avg_exprs_last_expression_bin

In [ ]:
rows_abs_diff = []

In [ ]:
from tqdm import tqdm

In [ ]:
%reload_ext memory_profiler

In [ ]:
def ANS_control_genes(ref_genes, avg_expr_sig_gene):
    ref_genes_means = avg_exprs_last_expression_bin[ref_genes]
    ref_genes_means = ref_genes_means.sort_values()
    rolled = ref_genes_means.rolling(ctrl_size, closed='right').mean()
    min_val_idx = np.argmin(((rolled - avg_expr_sig_gene).abs()))
    sig_gene_ctrl_genes = rolled.iloc[(min_val_idx - ctrl_size + 1):min_val_idx + 1]
    return avg_exprs_last_expression_bin[sig_gene_ctrl_genes.index].mean()

In [ ]:
Seurat_AG = []
for sig_gene in tqdm(sig_genes_to_test_for):
    
    ref_genes = list(set(avg_exprs_last_expression_bin.index).difference(set([sig_gene])))
    
#     avg_expr_sig_gene = avg_exprs_last_expression_bin[sig_gene]
    # Seurat_AG control genes
    Seurat_AG_mean_control = avg_exprs_last_expression_bin[ref_genes].mean()
#     Seurat_AG.append(np.abs(avg_expr_sig_gene - Seurat_AG_mean_control))
    Seurat_AG.append(Seurat_AG_mean_control)

In [ ]:
ANS = []
for sig_gene in tqdm(sig_genes_to_test_for):
    
    ref_genes = list(set(avg_exprs_last_expression_bin.index).difference(set([sig_gene])))
    
    avg_expr_sig_gene = avg_exprs_last_expression_bin[sig_gene]
    # ANS control genes
    ANS_mean_control = ANS_control_genes(ref_genes, avg_expr_sig_gene)
#     ANS.append(np.abs(avg_expr_sig_gene - ANS_mean_control))
    ANS.append(ANS_mean_control)    

In [ ]:
Seurat = []
for sig_gene in tqdm(sig_genes_to_test_for):
    
    ref_genes = list(set(avg_exprs_last_expression_bin.index).difference(set([sig_gene])))
    
#     avg_expr_sig_gene = avg_exprs_last_expression_bin[sig_gene]
    # Seurat et al. control genes 
    r_genes = ref_genes.copy()
    np.random.shuffle(r_genes)
    Seurat_mean_control = avg_exprs_last_expression_bin[r_genes[:ctrl_size]].mean()
#     Seurat.append(np.abs(avg_expr_sig_gene - Seurat_mean_control))
    Seurat.append(Seurat_mean_control)    

In [ ]:
adata.uns['log1p']['base']=None

In [ ]:
least_variable_genes_per_bin = get_least_variable_genes_per_bin_v1(
            adata,
            gene_bins,
            ctrl_size+1,
            method='seurat',
        )

In [ ]:
Seurat_LVG = []
for sig_gene in tqdm(sig_genes_to_test_for):
    
    ref_genes = list(set(avg_exprs_last_expression_bin.index).difference(set([sig_gene])))
    
#     avg_expr_sig_gene = avg_exprs_last_expression_bin[sig_gene]
    # LVCGS control genes 
    if len(set(least_variable_genes_per_bin[24]).difference([sig_gene]))==(ctrl_size+1):
        curr_contol = least_variable_genes_per_bin[24][0:-1]
    elif len(set(least_variable_genes_per_bin[24]).difference([sig_gene]))==(ctrl_size):
        curr_contol = list(set(least_variable_genes_per_bin[24]).difference([sig_gene]))
    else:
        raise ValueError
    Seurat_LVG_mean_control = avg_exprs_last_expression_bin[curr_contol].mean()
#     Seurat_LVG.append(np.abs(avg_expr_sig_gene - Seurat_mean_control))
    Seurat_LVG.append(Seurat_LVG_mean_control)

In [ ]:
df = pd.DataFrame({'ANS':ANS, 'Seurat':Seurat, 'Seurat_AG':Seurat_AG, 'Seurat_LVG':Seurat_LVG}, index = sig_genes_to_test_for)
if save:
    df.to_csv(os.path.join(storing_path, f'abs_diffs_control_to_sig_average_expression_{dataset}.csv'))

In [ ]:
df

In [ ]:
plt.figure(figsize=(10, 6))
avg_exprs_last_expression_bin[sig_genes_to_test_for].plot(label='average gene expression (AGE)', ls=':', dashes=(5, 1))
df['ANS'].plot(label='mean control values of ANS', ls=':', dashes=(5, 4))
df['Seurat'].plot(label='mean control values of Seurat', ls='--')
df['Seurat_AG'].plot(label='mean control values of Seurat_AG', ls='--')
df['Seurat_LVG'].plot(label='mean control values of Seurat_LVG', ls='--')
plt.legend(fontsize=14)
plt.title(f'Avg. gene expression and mean control values for each gene in last expression bin ({dataset.upper()})', fontsize=16);
plt.xlabel('Genes in last expression bin sorted by average expression.', fontsize=16);
plt.xticks((plt.gca().get_xticks()[[1, -2]]),
          [plt.gca().get_xticklabels()[1], plt.gca().get_xticklabels()[-2]], fontsize=16);
plt.ylabel('Log-Normalized expression.', fontsize=16);
plt.tight_layout()

if save:
    plt.savefig(os.path.join(storing_path, f'abs_diffs_control_to_sig_average_expression_{dataset}.png'), dpi=300)
    plt.savefig(os.path.join(storing_path, f'abs_diffs_control_to_sig_average_expression_{dataset}.svg'))

if show:
    plt.show()
else:
    plt.close()

In [ ]:
raise ValueError()

In [ ]:
# plt.figure(figsize=(10, 6))
# avg_exprs_last_expression_bin[sig_genes_to_test_for].plot(label='average gene expression (AGE)')
# df['ANS'].plot(label='abs. difference AGE and ANS control')
# df['Seurat'].plot(label='abs. difference AGE and Seurat control')
# df['Seurat_AG'].plot(label='abs. difference AGE and Seurat_AG control')
# df['Seurat_LVG'].plot(label='abs. difference AGE and Seurat_LVG control')
# plt.legend(fontsize=14)
# plt.title(f'Absolute distance between genes and controls for last expression bin ({dataset})', fontsize=16);
# plt.xlabel('Genes in last expression bin sorted by average expression.');
# plt.xticks((plt.gca().get_xticks()[[1, -2]]),
#           [plt.gca().get_xticklabels()[1], plt.gca().get_xticklabels()[-2]]);
# plt.tight_layout()
# plt.savefig(os.path.join(storing_path, f'abs_diffs_control_to_sig_average_expression_{dataset}.png'), dpi=300)
# plt.savefig(os.path.join(storing_path, f'abs_diffs_control_to_sig_average_expression_{dataset}.svg'))

In [ ]:
if save:
    df.describe().to_csv(os.path.join(storing_path, f'summarized_distances_{dataset}.csv'))